# Dataset Discovery and Exploration: State-of-the-art, Challenges and Opportunities
## Part 1: Dataset Search
### Framework Overview -- D3L


Our demo utilizes structured data derived from the Web Data Commons project, focusing on:
- **T2Dv2 Gold Standard for Matching Web Tables to DBpedia**: 108 tables from 9 entity classes. [Access here](https://webdatacommons.org/webtables/goldstandardV2.html).
- **Schema.org Table Corpus 2023**: 92 tables from 8 entity classes. [Access here](https://webdatacommons.org/structureddata/schemaorgtables/2023/index.html#toc3).
#### Input Dataset
The input dataset consists of structured data with various attributes. Below is a glimpse of the top 5 rows, showcasing the structure and type of data we are dealing with:

| Rank | Title                                | Category         | Publisher |
|------|--------------------------------------|------------------|-----------|
| 1    | Super Smash Bros. Melee              | Fighting         | Nintendo  |
| 2    | Pikmin 2                             | Strategy/Sim     | Nintendo  |
| 3    | Legend of Zelda: Collector's Edition | RPG              | Nintendo  |
| 4    | Legend of Zelda: The Wind Waker      | Action Adventure | Nintendo  |
| 5    | Metal Gear Solid: Twin Snakes        | Action Adventure | Konami    |



D3L utilizes a comprehensive approach based on:

1. **Attribute Header Similarity**
2. **Value Similarity**
3. **Format Similarity**
4. **Value Distribution**
5. **Attribute value embeddings**
#### Output Datasets: Top k searched dataset results



In [1]:
## Download required words
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Autoload all modules
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### Generate LSH indexes for all evidence in D3L

In [2]:
from Utils import mkdir

# Import and initialize modules
from d3l.indexing.similarity_indexes import NameIndex, FormatIndex, ValueIndex, EmbeddingIndex, DistributionIndex
from d3l.input_output.dataloaders import CSVDataLoader
from d3l.querying.query_engine import QueryEngine
from d3l.utils.functions import pickle_python_object, unpickle_python_object
import os
import pandas as pd

data_path = "Datasets"
result_path = "Result/"
threshold = 0.5
mkdir(result_path)

dataloader = CSVDataLoader(
        root_path=data_path,
        encoding='utf-8'
)


##### Generating/loading NameIndex of tables
Name index: Use q-gram analysis of attribute names to calculate the Jaccard distance between their qsets.

In [3]:
name_lsh = os.path.join(result_path, 'Name.lsh')
print(name_lsh)
if os.path.isfile(name_lsh):
    name_index = unpickle_python_object(name_lsh)
    print("Name LSH index: LOADED!")
else:
    name_index = NameIndex(dataloader=dataloader, index_similarity_threshold=threshold)
    pickle_python_object(name_index, name_lsh)
    print("Name LSH index: SAVED!")

Result/Name.lsh
Name LSH index: LOADED!


##### Generating/loading FormatIndex of tables
 Format Index: Identifies data formats through regular expressions

In [4]:
format_lsh = os.path.join(result_path, './format.lsh')
if os.path.isfile(format_lsh):
    format_index = unpickle_python_object(format_lsh)
    print("Format LSH index: LOADED!")
else:
    format_index = FormatIndex(dataloader=dataloader, index_similarity_threshold=threshold)
    pickle_python_object(format_index, format_lsh)
    print("Format LSH index: SAVED!")

Format LSH index: LOADED!


##### Generating/loading ValueIndex of tables
Value Index: Employs TFIDF tokens to represent values, with Jaccard distance between their t-sets assessing similarity.

In [5]:
value_lsh = os.path.join(result_path, './value.lsh')
if os.path.isfile(value_lsh):
    value_index = unpickle_python_object(value_lsh)
    print("Value LSH index: LOADED!")
else:
    value_index = ValueIndex(dataloader=dataloader, index_similarity_threshold=threshold)
    pickle_python_object(value_index, value_lsh)
    print("Value LSH index: SAVED!")

Value LSH index: LOADED!


##### Generating/loading DistributionIndex of tables
Distribution Index: Assesses numeric attribute value relatedness via the Kolmogorov-Smirnov statistic, offering insights into domain-originating samples.

In [6]:
distribution_lsh = os.path.join(result_path, './distribution.lsh')
if os.path.isfile(distribution_lsh):
    distribution_index = unpickle_python_object(distribution_lsh)
    print("Distribution LSH index: LOADED!")
else:
    distribution_index = DistributionIndex(dataloader=dataloader, index_similarity_threshold=threshold)
    pickle_python_object(distribution_index, distribution_lsh)
    print("Distribution LSH index: SAVED!")

Distribution LSH index: LOADED!


##### Generating/loading EmbeddingIndex of tables
Embedding index: Determines textual content relatedness through cosine distance of their vector representations.

In [7]:
embedding_lsh = os.path.join(result_path, './embedding.lsh')
if os.path.isfile(embedding_lsh):
    embedding_index = unpickle_python_object(embedding_lsh)
    print("Embedding LSH index: LOADED!")
else:
    embedding_index = EmbeddingIndex(dataloader=dataloader,
                                     index_similarity_threshold=threshold)
    pickle_python_object(embedding_index, embedding_lsh)
    print("Embedding LSH index: SAVED!")


File exists. Use --overwrite to download anyway.
Loading embeddings. This may take a few minutes ...
Embedding LSH index: LOADED!


##### show the input table

In [8]:
searched_table = 'file_de2e4073-570f-4b79-bb7a-7d3dfec1c238'
table_df = dataloader.read_table(searched_table)
print(table_df.head(5))
df = pd.read_csv("groundTruth.csv")

                      OID Tipo Organismo  INCISO  UE  \
0  2.16.858.0.0.0.1.1.9.0     Ministerio       8   1   
1  2.16.858.0.0.0.1.1.9.0     Ministerio       8   1   
2  2.16.858.0.0.0.1.1.9.0     Ministerio       8   1   
3  2.16.858.0.0.0.1.1.9.0     Ministerio       8   1   
4  2.16.858.0.0.0.1.1.9.0     Ministerio       8   1   

                                Nombre Inciso  \
0  Ministerio de Industria, Energía y Minería   
1  Ministerio de Industria, Energía y Minería   
2  Ministerio de Industria, Energía y Minería   
3  Ministerio de Industria, Energía y Minería   
4  Ministerio de Industria, Energía y Minería   

                         Nombre UE  \
0  Dirección General de Secretaría   
1  Dirección General de Secretaría   
2  Dirección General de Secretaría   
3  Dirección General de Secretaría   
4  Dirección General de Secretaría   

                             Nombre Organismo                 ID UNIDAD  \
0  Ministerio de Industria, Energía y Minería  2.16.858.0.0.0.1.

Query table in the framework using all the above indexes

In [9]:
# Searched results, K =10
qe = QueryEngine(name_index, value_index, embedding_index, format_index,  distribution_index)
results, extended_results = qe.table_query(table=dataloader.read_table(table_name=searched_table),
                                           aggregator=None, k=10, verbose=True)
print(results)

[('file_de2e4073-570f-4b79-bb7a-7d3dfec1c238', [1.0, 0.4444444444444444, 0.34551950133215587, 0.8617621527777778, 0.1111111111111111]), ('file_798f5e29-af23-47cc-8e9e-7fe7abb9484e', [1.0, 0.0, 0.17522335456239618, 0.06259066922821083, 0.04361398415872039]), ('file_212354b6-72dd-427c-9fcd-4b06cf21606d', [1.0, 0.0, 0.15531639696695243, 0.06259066922821083, 0.04361398415872039]), ('file_9694cce8-ca17-40f5-aee8-ffd4144290bc', [1.0, 0.0, 0.15346569722368994, 0.06259066922821083, 0.04361398415872039]), ('file_892da481-d210-479c-9b5a-1c7f82d387dc', [1.0, 0.0, 0.1384129271104965, 0.17088403562678292, 0.04361398415872039]), ('file_11238d12-4fa0-4c89-9f8f-075f33ae373e', [1.0, 0.0, 0.12687984925587995, 0.06259066922821083, 0.04361398415872039]), ('file_b8cba44e-f5e3-48a8-868f-ba7b21b5c5d3', [0.9926006610576922, 0.0, 0.23065613554666214, 0.03196572393515189, 0.04361398415872039]), ('file_03d299c4-e8e0-45f0-9206-81984ca7b439', [0.9926006610576922, 0.0, 0.18336539907539479, 0.06259066922821083, 0.04

##### Output the results and check if the output tables have the same type as the input query table. This is a validation step that checks against the groundTruth, to see if the classes found match.

In [10]:
# Summarize searched results in a table

# class_input_table = df[df['fileName'] == searched_table+".csv"]['class'].tolist()[0]

# data = []
# exceptions = []
# average = []
# for table, score in results:
#         print(table)
#         print(score)
#         class_table = df[df['fileName'] == table+".csv"]['class'].tolist()
#         if len(class_table)==0:
#             class_table = "No Class found"
#         else:
#             class_table = class_table[0]
#         data.append((table, score,class_table))
#         average.append(sum(score)/len(score))
#         if class_table!=class_input_table:
#             exceptions.append(table)

# Creating the DataFrame

# result_summarization = pd.DataFrame(data, columns=["Table Name", "Scores", "Ground Truth Class"])
# result_summarization = pd.concat([result_summarization.drop(["Scores"], axis=1), result_summarization["Scores"].apply(pd.Series)], axis=1).round(3)
# result_summarization.columns = ["Table Name", "Class", "Header Score", "Value Score", "Embedding Score","Format Score",  "Distribution Score"]
# result_summarization["average score"] = average
# print(result_summarization)
# print(result_summarization["Table Name"])

##### For tables that does not belong to the same class of input table, show the specific table.

In [11]:
# for table_name in exceptions:
#     table_except = dataloader.read_table(table_name)
#     table_except_part = table_except.head(5)
#     print(table_except_part)
#     break

##### Individual search using different methods

In [12]:
# Individual search results
# Name index query
topk = 10
def remove_search_col(listA, check_col):
    return [i for i, score in listA if i!=check_col]
        
def check_column(Dataloader:CSVDataLoader, combined_column_name):
    table_name, column_name = combined_column_name.split(".")
    table = Dataloader.read_table(table_name)
    return table[column_name]

def table_results(list_result):
    return pd.DataFrame(list_result, columns=["Column Name", "Scores"])

name_results = name_index.query(query="INCISO", k=topk)
print(f"Name results are \n {table_results(name_results)} \n")

Name results are 
                                         Column Name  Scores
0  file_4b58569a-0e04-4777-8fee-57a152d4340f.INCISO     1.0
1  file_212354b6-72dd-427c-9fcd-4b06cf21606d.INCISO     1.0
2  file_a87ace2f-b75b-4406-b813-c7d21771db03.INCISO     1.0
3  file_03d299c4-e8e0-45f0-9206-81984ca7b439.INCISO     1.0
4  file_892da481-d210-479c-9b5a-1c7f82d387dc.Inciso     1.0
5  file_c16f6539-0963-42d9-9a2e-6210c37a3ad4.Inciso     1.0
6  file_11238d12-4fa0-4c89-9f8f-075f33ae373e.INCISO     1.0
7  file_798f5e29-af23-47cc-8e9e-7fe7abb9484e.INCISO     1.0
8  file_9694cce8-ca17-40f5-aee8-ffd4144290bc.INCISO     1.0
9  file_de2e4073-570f-4b79-bb7a-7d3dfec1c238.INCISO     1.0 



In [13]:
### Value 
## Currently not working. Commented to be able to run "all above cells" without interruptions.
## TODO: Fix value index

# Value index query
value_results = value_index.query(query=table_df["Nombre Organismo"], k=topk)
print(f"Value results are \n {value_results}\n")
columns = [check_column(dataloader, column) for column,score in value_results if column !="file_de2e4073-570f-4b79-bb7a-7d3dfec1c238.Nombre Organismo" ]
print(f"Value indexes results are \n {table_results(value_results)}\n")
if(columns):
    print(f"example results searching Attribute value indexes:\n {columns[0]} \n")

In [14]:
# Embeddings index 
embedding_results = embedding_index.query(query=table_df["CORREO INSTITUCIONAL"], k=topk)
print(f"Embedding indexes results are \n{table_results(embedding_results)} \n")
embedding_column  = [check_column(dataloader, column) for column,score in embedding_results if column !="file_de2e4073-570f-4b79-bb7a-7d3dfec1c238.CORREO INSTITUCIONAL" ]
if(embedding_column):
    print(f"example results searching embedding value indexes:\n {embedding_column[0]} \n")

Embedding indexes results are 
                                         Column Name    Scores
0  file_de2e4073-570f-4b79-bb7a-7d3dfec1c238.CORR...  0.738281
1  file_212354b6-72dd-427c-9fcd-4b06cf21606d.Nomb...  0.687500
2  file_a87ace2f-b75b-4406-b813-c7d21771db03.Nive...  0.659180
3  file_a87ace2f-b75b-4406-b813-c7d21771db03.ENLA...  0.651367
4  file_b8cba44e-f5e3-48a8-868f-ba7b21b5c5d3.ENLA...  0.689453
5  file_11238d12-4fa0-4c89-9f8f-075f33ae373e.Nomb...  0.677734
6  file_b8cba44e-f5e3-48a8-868f-ba7b21b5c5d3.Nomb...  0.669922
7  file_798f5e29-af23-47cc-8e9e-7fe7abb9484e.Nomb...  0.665039
8  file_11238d12-4fa0-4c89-9f8f-075f33ae373e.Nive...  0.635742
9  file_c16f6539-0963-42d9-9a2e-6210c37a3ad4.ID P...  0.647461 

example results searching embedding value indexes:
 0         Intendente Departamental
1               Secretario General
2               Secretaría Privada
3                        Asesoría 
4                        Asesoría 
                  ...             
120    Junta

## Part 2: Dataset Navigation
### Framework Overview -- Aurum
This is a simplified version of Aurum. It includes two phases: signature building stage and relationship building stage.
Signatures: LSH indexes from D3L: name index and value index
Relationship Building Stage: Search similar columns based on similarity in name and value LSH indexes.


#### Prerequisites: detect subject columns and type of columns

In [15]:
import pickle
from TableMiner.SCDection.TableAnnotation import TableColumnAnnotation as TA
"""
Find the column type and Named entity scores in each table,
 store the table and related column type/NE-scores info as dict in pickle file
"""
def subjectColDetection(DATA_PATH, RESULT_PATH):
    table_dict = {}
    # Try to load the dict from pickle file
    if "dict.pkl" in os.listdir(RESULT_PATH):
        with open(os.path.join(RESULT_PATH,"dict.pkl"), "rb") as f:
            table_dict = pickle.load(f)
    else:
        table_names = os.listdir(DATA_PATH)
        print(table_names)
        for tableName in table_names:
            table_dict[tableName] = []
            table = pd.read_csv(f"Datasets/{tableName}")
            try:
                annotation_table = TA(table, SearchingWeb = False)
                annotation_table.subcol_Tjs()
                print(annotation_table)
                table_dict[tableName].append(annotation_table.annotation)
                table_dict[tableName].append(annotation_table.column_score)
            except Exception as e:
                print(f"Error in {tableName} : {e}")
                continue
        # Save the dict as pickle file
        with open(os.path.join(RESULT_PATH, "dict.pkl"), "wb") as save_file:
            pickle.dump(table_dict, save_file)
    return table_dict

# Perform the call to the method
SubjectCol_dict = subjectColDetection(data_path, "Result")
print(SubjectCol_dict)

{'file_03d299c4-e8e0-45f0-9206-81984ca7b439.csv': [{0: <ColumnType.other: 5>, 1: <ColumnType.long_text: 0>, 2: <ColumnType.number: 2>, 3: <ColumnType.number: 2>, 4: <ColumnType.named_entity: 1>, 5: <ColumnType.named_entity: 1>, 6: <ColumnType.named_entity: 1>, 7: <ColumnType.number: 2>, 8: <ColumnType.named_entity: 1>, 9: <ColumnType.other: 5>, 10: <ColumnType.named_entity: 1>, 11: <ColumnType.named_entity: 1>, 12: <ColumnType.number: 2>, 13: <ColumnType.named_entity: 1>, 14: <ColumnType.named_entity: 1>, 15: <ColumnType.number: 2>, 16: <ColumnType.named_entity: 1>, 17: <ColumnType.other: 5>}, {4: 0.8944271909999157, 5: -0.4999999999999998, 6: -0.7559289460184544, 8: -0.7135060680126758, 10: -0.6321943486518861, 11: -0.59837350163391, 13: -0.543045080249949, 14: -0.5202551461131063, 16: -0.48186695704599686}], 'file_06012849-ab0a-4b60-b9af-82386fccbf18.csv': [{0: <ColumnType.other: 5>, 1: <ColumnType.long_text: 0>, 2: <ColumnType.number: 2>, 3: <ColumnType.number: 2>, 4: <ColumnType.na

##### Find the subject columns of result tables from Part I Dataset search.

In [16]:
result_tables = os.listdir(data_path)
subject_columns=[]
all_columns = []
print(result_tables)

"""
Use iteration and the above column info dict to find the subject columns (and all columns)
 in each table.
"""
for table in result_tables:
    df_table = dataloader.read_table(table[:-4])
    annotation, NE_column_score = SubjectCol_dict[table]
    max_score = max(NE_column_score.values())
    all_columns.extend([f"{table[:-4]}.{df_table.columns[i]}" for i in NE_column_score.keys()])
    subcol_index = [key for key, value in NE_column_score.items() if value == max_score]
    for index in subcol_index:
        subject_columns.append(f"{table[:-4]}.{df_table.columns[index]}")
print(subject_columns)

['file_03d299c4-e8e0-45f0-9206-81984ca7b439.csv', 'file_06012849-ab0a-4b60-b9af-82386fccbf18.csv', 'file_11238d12-4fa0-4c89-9f8f-075f33ae373e.csv', 'file_212354b6-72dd-427c-9fcd-4b06cf21606d.csv', 'file_4b58569a-0e04-4777-8fee-57a152d4340f.csv', 'file_61d17612-2499-49d6-beda-4feab750b8e7.csv', 'file_798f5e29-af23-47cc-8e9e-7fe7abb9484e.csv', 'file_892da481-d210-479c-9b5a-1c7f82d387dc.csv', 'file_92bc37e9-42db-4d2d-afbc-a9442e7052c3.csv', 'file_9694cce8-ca17-40f5-aee8-ffd4144290bc.csv', 'file_a87ace2f-b75b-4406-b813-c7d21771db03.csv', 'file_b8cba44e-f5e3-48a8-868f-ba7b21b5c5d3.csv', 'file_c16f6539-0963-42d9-9a2e-6210c37a3ad4.csv', 'file_c953437a-0a61-4d8b-bd46-fec567e0780e.csv', 'file_de2e4073-570f-4b79-bb7a-7d3dfec1c238.csv']
['file_03d299c4-e8e0-45f0-9206-81984ca7b439.Nombre Inciso', 'file_06012849-ab0a-4b60-b9af-82386fccbf18.NombreInciso', 'file_11238d12-4fa0-4c89-9f8f-075f33ae373e.Nombre Inciso', 'file_212354b6-72dd-427c-9fcd-4b06cf21606d.Nombre Inciso', 'file_4b58569a-0e04-4777-8fe

In [17]:
from Aurum.graph import buildGraph,draw_interactive_network
# Use Aurum to build the graph
# TODO: Evaluate the possiblity of using other indexes as well.
aurum_graph = buildGraph(dataloader, data_path, [name_index, value_index], target_path="Result", table_dict=SubjectCol_dict)
import networkx as nx

"""
Find the subgraph in the Aurum that contains the provided nodes and all the nodes that
have routine to these nodes
"""
def subgraph(given_nodes, graph: nx.Graph()):
    # Find the connected components containing the given node
    subgraphs = list(nx.connected_components(graph))
    relevant_nodes = set()
    for node in given_nodes:
        for sg in subgraphs:
            if node in sg:
                relevant_nodes.update(sg)
    new_graph = aurum_graph.subgraph(relevant_nodes).copy()
    return new_graph
subject_columns_graph = subgraph(subject_columns, aurum_graph)
result_SC_graph = subgraph(subject_columns, aurum_graph)
draw_interactive_network(result_SC_graph)


In [18]:
# See all columns in the graph
result_graph = subgraph(all_columns, aurum_graph)
draw_interactive_network(result_graph)

## Part 3: Dataset Annotation
### Framework Overview -- TableMiner+
#### Input dataset: 13 tables from 13 domain, while each domain has 1 table
The 13 domains include:
1. **Airport**
2. **City**
3. **CollegeOrUniversity**
4. **Company**
5. **Continent**
6. **Country**
7. **Hospital**
8. **LandmarksOrHistoricalBuildings**
9. **Monarch**
10. **Movie**
11. **Museum**
12. **Scientist**
13. **VideoGame**

TableMiner+ has 4 steps:
1. **Subject Column Detection: Including column (data) type detection** 
2. **NE-Column interpretation - the LEARNING phase:**
***2.1 preliminary cell annotation***
***2.2 column semantic type annotation***
***2.3 property annotation***
3. **NE-Column interpretation - the UPDATE phase: revise annotation until all annotation is stabilized**
4. **Relation enumeration and annotating literal-columns(not included yet)**

##### show the example annotation table

In [19]:
import pandas as pd
from TableMiner.LearningPhase.Update import TableLearning,  updatePhase
from TableMiner.SearchOntology import SearchDBPedia

# The map removes .csv from the table names
table_domains = os.listdir(data_path)
for table in table_domains:
    table_domains[table_domains.index(table)] = table[:-4]
print(table_domains)

['file_03d299c4-e8e0-45f0-9206-81984ca7b439', 'file_06012849-ab0a-4b60-b9af-82386fccbf18', 'file_11238d12-4fa0-4c89-9f8f-075f33ae373e', 'file_212354b6-72dd-427c-9fcd-4b06cf21606d', 'file_4b58569a-0e04-4777-8fee-57a152d4340f', 'file_61d17612-2499-49d6-beda-4feab750b8e7', 'file_798f5e29-af23-47cc-8e9e-7fe7abb9484e', 'file_892da481-d210-479c-9b5a-1c7f82d387dc', 'file_92bc37e9-42db-4d2d-afbc-a9442e7052c3', 'file_9694cce8-ca17-40f5-aee8-ffd4144290bc', 'file_a87ace2f-b75b-4406-b813-c7d21771db03', 'file_b8cba44e-f5e3-48a8-868f-ba7b21b5c5d3', 'file_c16f6539-0963-42d9-9a2e-6210c37a3ad4', 'file_c953437a-0a61-4d8b-bd46-fec567e0780e', 'file_de2e4073-570f-4b79-bb7a-7d3dfec1c238']


##### Perform NE-Column interpretation (Table Learning includes the process of subject column detection of a table)

In [20]:
def table_annotation(tableName, subcol_dict):
    tableD = dataloader.read_table(tableName)
    print(tableD)
    annotation_table, NE_Score = subcol_dict[tableName + ".csv"]
    print(annotation_table)
    ### Learning phase of TableMiner+
    tableLearning = TableLearning(tableD, NE_column=NE_Score)
    ### Perform NE-Column interpretation - the UPDATE phase
    print("starting learning phase")
    tableLearning.table_learning()
    print("starting update phase")
    updatePhase(tableLearning)
    return tableLearning

In [21]:
import json

# Codigo duplicado. Es para imprimir lindo el json de las requests.
def pretty_print_json(loaded_json):
    print(json.dumps(loaded_json, indent=2, ensure_ascii=False))

# Mergea dos diccionarios.
# Los valores de dict1 sobreescriben los valores de dict2 en caso de colision
def merge_dicts(dict1, dict2):
    return {**dict2, **dict1}

# Agrega las requests guardadas en el archivo pickle al diccionario de requests de SearchDBPedia
# No se remueven los valores en memoria dinamica.
# Los valores predominantes son los de SearchDBPedia.
def load_ontology_requests(dict_path, dict_name):
    target_file = os.path.join(dict_path, dict_name)
    with open(target_file, 'rb') as file:
        request_cache = pickle.load(file)
        SearchDBPedia.searches_dictionary = merge_dicts(request_cache['searches'], SearchDBPedia.searches_dictionary)
        SearchDBPedia.retrieve_entity_triples_dictionary = merge_dicts(request_cache['retrieve_entity_triples'], SearchDBPedia.retrieve_entity_triples_dictionary)
        SearchDBPedia.retrieve_concepts_dictionary = merge_dicts(request_cache['retrieve_concepts'], SearchDBPedia.retrieve_concepts_dictionary)
        SearchDBPedia.retrieve_concept_uri_dictionary = merge_dicts(request_cache['get_concept_uri'], SearchDBPedia.retrieve_concept_uri_dictionary)
        SearchDBPedia.retrieve_definitional_sentence_dictionary = merge_dicts(request_cache['get_definitional_sentence'], SearchDBPedia.retrieve_definitional_sentence_dictionary)
    return request_cache

request_cache = load_ontology_requests("Result", "ontologyRequests.pkl")    
pretty_print_json(request_cache['searches'])

In [26]:
# Learning phase for the selected table.

searched_table = table_domains[4]
learning = table_annotation(searched_table, SubjectCol_dict)

                OID Tipo Organismo  INCISO  UE              Nombre Inciso  \
0  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   
1  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   
2  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   
3  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   
4  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   

                   Nombre UE           Nombre Organismo  ID UNIDAD  \
0  Banco Central del Uruguay  Banco Central del Uruguay          1   
1  Banco Central del Uruguay  Banco Central del Uruguay          1   
2  Banco Central del Uruguay  Banco Central del Uruguay          1   
3  Banco Central del Uruguay  Banco Central del Uruguay          2   
4  Banco Central del Uruguay  Banco Central del Uruguay          3   

  Tipo Unidad  Programa o Proyecto Nombre Unidad           Nivel jerárquico  \
0  Estructura                  NaN   

In [29]:
from TableMiner.SearchOntology import SearchDBPedia

print("Network Calls")
print("Amount of searches", SearchDBPedia.amount_of_search)
print("Amount of unique searches", SearchDBPedia.unique_searches.__len__(), "\n", SearchDBPedia.unique_searches, "\n")

print("Amount of retrieve entity triples", SearchDBPedia.amount_of_retrieve_entity_triples)
print("Amount of unique entity triples", SearchDBPedia.unique_retrieve_entity_triples.__len__(), "\n", SearchDBPedia.unique_retrieve_entity_triples, "\n")

print("Amount of retrieve concepts", SearchDBPedia.amount_of_retrieve_concepts)
print("Amount of unique concepts", SearchDBPedia.unique_retrieve_concepts.__len__(), "\n", SearchDBPedia.unique_retrieve_concepts, "\n")

print("Amount of concept uri", SearchDBPedia.amount_of_get_concept_uri)
print("Amount of unique concept uri", SearchDBPedia.unique_get_concept_uri.__len__(), "\n", SearchDBPedia.unique_get_concept_uri, "\n")

print("Amount of definitional sentences", SearchDBPedia.amount_of_get_definitional_sentence)
print("Amount of unique definitional sentences", SearchDBPedia.unique_get_definitional_sentence.__len__(), "\n", SearchDBPedia.unique_get_definitional_sentence, "\n")

Network Calls
Amount of searches 111
Amount of unique searches 13 
 {'LICANDRO FERRANDO, GERARDO MARCELO', 'BERTI MOYANO, IGNACIO', 'Estructura', 'info@bcu.gub.uy', 'GERENCIA DE LR', 'PRESIDENTE DEL BANCO', 'VICE PRESIDENTE DEL BANCO', 'ARTECONA GULLA, DANIEL EDUARDO', 'DIRECTOR DEL BANCO', 'Banco Central del Uruguay', 'RIBEIRO TORRADO, WASHINGTON RAUL', 'DIAGONAL FABINI 777', 'LABAT LEGARRA, DIEGO'} 

Amount of retrieve entity triples 234
Amount of unique entity triples 1 
 {'http://dbpedia.org/resource/Central_Bank_of_Uruguay'} 

Amount of retrieve concepts 180
Amount of unique concepts 1 
 {'http://dbpedia.org/resource/Central_Bank_of_Uruguay'} 

Amount of concept uri 135
Amount of unique concept uri 15 
 {'Abstraction100002137', 'YagoLegalActor', 'WikicatCentralBanks', 'Company108058098', 'WikicatBanksEstablishedIn1967', 'Thing', 'Group100031264', 'WikicatBanksOfUruguay', 'Organization108008335', 'WikicatGovernment-ownedCompaniesOfUruguay', 'SpatialThing', 'Institution108053576', '

In [30]:
# Learning phase for the selected table.

searched_table = table_domains[4]
learning = table_annotation(searched_table, SubjectCol_dict)

                OID Tipo Organismo  INCISO  UE              Nombre Inciso  \
0  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   
1  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   
2  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   
3  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   
4  2.16.858.0.0.1.3  Ente autónomo      50   1  Banco Central del Uruguay   

                   Nombre UE           Nombre Organismo  ID UNIDAD  \
0  Banco Central del Uruguay  Banco Central del Uruguay          1   
1  Banco Central del Uruguay  Banco Central del Uruguay          1   
2  Banco Central del Uruguay  Banco Central del Uruguay          1   
3  Banco Central del Uruguay  Banco Central del Uruguay          2   
4  Banco Central del Uruguay  Banco Central del Uruguay          3   

  Tipo Unidad  Programa o Proyecto Nombre Unidad           Nivel jerárquico  \
0  Estructura                  NaN   

In [ ]:
print("Network Calls")
print("Amount of searches", SearchDBPedia.amount_of_search)
print("Amount of unique searches", SearchDBPedia.unique_searches.__len__(), "\n", SearchDBPedia.unique_searches, "\n")

print("Amount of retrieve entity triples", SearchDBPedia.amount_of_retrieve_entity_triples)
print("Amount of unique entity triples", SearchDBPedia.unique_retrieve_entity_triples.__len__(), "\n", SearchDBPedia.unique_retrieve_entity_triples, "\n")

print("Amount of retrieve concepts", SearchDBPedia.amount_of_retrieve_concepts)
print("Amount of unique concepts", SearchDBPedia.unique_retrieve_concepts.__len__(), "\n", SearchDBPedia.unique_retrieve_concepts, "\n")

print("Amount of concept uri", SearchDBPedia.amount_of_get_concept_uri)
print("Amount of unique concept uri", SearchDBPedia.unique_get_concept_uri.__len__(), "\n", SearchDBPedia.unique_get_concept_uri, "\n")

print("Amount of definitional sentences", SearchDBPedia.amount_of_get_definitional_sentence)
print("Amount of unique definitional sentences", SearchDBPedia.unique_get_definitional_sentence.__len__(), "\n", SearchDBPedia.unique_get_definitional_sentence, "\n")

In [41]:
def store_learning(table, learning, dict_path, dict_name):
    target_file = os.path.join(dict_path, dict_name)
    if os.path.isfile(target_file):
        with open(target_file, 'rb') as file:
            dict_annotation = pickle.load(file)
    else:
        dict_annotation = {}
    dict_annotation[table] = learning
    with open(target_file, 'wb') as file:
        pickle.dump(dict_annotation, file)

store_learning(searched_table, learning, "Result", "annotationDict.pkl")

In [41]:
# Guarda las requests cacheadas de la Ontologia en un archivo pickle
# Obtiene las que estan guardadas hasta el momento y le suma las nuevas
def store_ontology_requests(dict_path, dict_name):
    target_file = os.path.join(dict_path, dict_name)
    
    saved_requests = load_ontology_requests(dict_path, dict_name)
    request_caching = {}
    request_caching['searches'] = merge_dicts(SearchDBPedia.searches_dictionary, saved_requests['searches'])
    request_caching['retrieve_entity_triples'] = merge_dicts(SearchDBPedia.retrieve_entity_triples_dictionary, saved_requests['retrieve_entity_triples'])
    request_caching['retrieve_concepts'] = merge_dicts(SearchDBPedia.retrieve_concepts_dictionary, saved_requests['retrieve_concepts'])
    request_caching['get_concept_uri'] = merge_dicts(SearchDBPedia.retrieve_concept_uri_dictionary, saved_requests['get_concept_uri'])
    request_caching['get_definitional_sentence'] = merge_dicts(SearchDBPedia.retrieve_definitional_sentence_dictionary, saved_requests['get_definitional_sentence'])
    with open(target_file, 'wb') as file:
        pickle.dump(request_caching, file)

store_ontology_requests("Result", "ontologyRequests.pkl")

In [ ]:
def findAnnotation(dict_of_annotation,tableN):
    learningT = dict_of_annotation[tableN]
    annotation_class = learningT.get_annotation_class()
    for columnIndex, learning_class in annotation_class.items():
        tableDataframe = dataloader.read_table(tableN)
        column = tableDataframe.iloc[:,columnIndex]
        cellAnnotation  = learning_class.get_cell_annotation()[:5]
        ColumnSemantics = learning_class.get_winning_concepts()
        df_t = pd.concat([column[:5], cellAnnotation], axis=1)
        print(f"column and Cell annotation of the column:\n{df_t}\n")
        print(f"Column {column.name} semantic type: {ColumnSemantics}")


with open("Result/annotationDict.pkl", 'rb') as file:
    dict_annotation = pickle.load(file)
    
findAnnotation(dict_annotation, searched_table)

##### Check the tables' subject column annotations

## Part 4: Schema Inference
### Framework Overview -- Starmie
#### Input dataset: all 200 tables covering 13 specific domains
Embedding methods: Starmie
Table class inference method: Hierarchical clustering
Similarity metric: Average column embeddings of each table
Type of result table clusters: the most frequently appeared class in the ground truth in each cluster


##### Use clustering on table's embeddings to detect the types/domains' of tables

In [ ]:
import clustering  as c
"""
Generation of Embeddings: Please run the ./starmie/cmd.sh script to generate the tables' embeddings 
We uploaded the sample embeddings generated by this script to the Result path.
"""
clustering_result = c.typeInference("Result/tableEmbeddings.pkl", "Agglomerative", numEstimate=13)
index_cluster = [i for i, tables in clustering_result.items() if "T2DV2_122.csv" in tables]
# The overall result of clustering in a dataframe, includes cluster id, GT label, size, precision,
# Ranked by precision
result_precision = c.result_precision(clustering_result)
print(result_precision)

In [ ]:
# Check the last second clusters' score
print("\n",result_precision.iloc[-2])
# Check inside the cluster, what kind of table does it contain
checked_cluster = clustering_result[result_precision.iloc[-2]["cluster id"]]
innerInfo = c.inner_cluster(checked_cluster)
print(innerInfo)
# select random two tables inside the sample cluster; check and compare their column headers
# to find the difference/similarities between then
rows = c.sample_tables_cluster(innerInfo)
first_table = dataloader.read_table(rows.iloc[0, 0][:-4])
second_table = dataloader.read_table(rows.iloc[1, 0][:-4])
print(first_table.columns,"\n",second_table.columns)

##### Use clustering on column embeddings to detect column's type
column type inference: using hierarchical clustering on the column embeddings


In [ ]:
import os
### Cluster all column embeddings under specified domain
column_clustering = c.conceptualAttri(os.path.join(c.current_dir_path, "Datasets"),
                os.path.join(c.current_dir_path, "Result/tableEmbeddings.pkl"),
                clustering_method="Agglomerative",
                domain="VideoGame",
                numEstimate=13)

check_table = "T2DV2_122"

for column_index, column_clusters in column_clustering.items():
    for i in  column_clusters:
        if check_table in i:
            column = [i for i in column_clusters if check_table in i][0]
            print(f"Cluster has {column} \n",column_clusters, "\n")
